<a href="https://colab.research.google.com/github/Bahae9/Kaggle_competitions/blob/main/Regression_of_Used_Car_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c playground-series-s4e9
!unzip playground-series-s4e9.zip

  0% 0.00/7.84M [00:00<?, ?B/s]
100% 7.84M/7.84M [00:00<00:00, 84.6MB/s]
Archive:  playground-series-s4e9.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install catboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow import keras
from keras import layers
import optuna

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
df = pd.read_csv('/content/train.csv')
df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900


In [ ]:
for column in df.columns:
  print(column, len(df[column].unique()))

id 188533
brand 57
model 1897
model_year 34
milage 6651
fuel_type 8
engine 1117
transmission 52
ext_col 319
int_col 156
accident 3
clean_title 2
price 1569


In [ ]:
df.isna().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [ ]:
def encode(df):
    df = df.drop(columns=['model', 'engine', 'brand', 'ext_col'])
    df['clean_title'] = df['clean_title'].fillna('No')
    df['model_year'] = df['model_year'] - 1973
    df = pd.get_dummies(df, columns=['fuel_type', 'transmission', 'int_col', 'accident'], prefix=['fuel_type', 'transmission', 'int_col', 'accident'])
    mapping = {False: 0, True: 1, 'Yes': 1, 'No': 0}
    df = df.replace(mapping)
    return df

In [ ]:
df = df.dropna()
# df['clean_title'] = df['clean_title'].fillna('No')
# for column in df.columns:
#   if df[column].dtype == 'object':
#     df[column].fillna(df[column].mode()[0], inplace=True)
#   else:
#     df[column].fillna(df[column].mean(), inplace=True)
df = encode(df)
df.head()

,id,model_year,milage,clean_title,price,fuel_type_Diesel,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,...,int_col_Very Light Cashmere,int_col_WHITE,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_At least 1 accident or damage reported,accident_None reported
0,0,34,213000,1,4200,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,29,143250,1,4999,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,29,136731,1,13900,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,44,19500,1,45000,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,48,7388,1,97500,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
df.describe()

,id,model_year,milage,clean_title,price,fuel_type_Diesel,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,...,int_col_Very Light Cashmere,int_col_WHITE,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_At least 1 accident or damage reported,accident_None reported
count,162610.000000,162610.000000,162610.000000,162610.0,1.626100e+05,162610.000000,162610.000000,162610.000000,162610.000000,162610.000000,...,162610.000000,162610.000000,162610.000000,162610.000000,162610.000000,162610.000000,162610.000000,162610.000000,162610.000000,162610.000000
mean,94197.039137,42.158822,71105.487086,1.0,4.081699e+04,0.022459,0.032194,0.906445,0.031124,0.003044,...,0.000043,0.000018,0.000141,0.000025,0.021911,0.000344,0.000314,0.021807,0.247211,0.752789
std,54467.770815,5.699531,50157.874111,0.0,7.653857e+04,0.148170,0.176514,0.291210,0.173652,0.055089,...,0.006561,0.004295,0.011892,0.004960,0.146394,0.018554,0.017707,0.146053,0.431392,0.431392
min,0.000000,1.000000,100.000000,1.0,2.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47026.250000,39.000000,29000.000000,1.0,1.570000e+04,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,94139.000000,43.000000,64300.000000,1.0,2.850000e+04,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,141421.750000,46.000000,101000.000000,1.0,4.699900e+04,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,188532.000000,51.000000,405000.000000,1.0,2.954083e+06,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162610 entries, 0 to 188532
Columns: 222 entries, id to accident_None reported
dtypes: int64(222)
memory usage: 276.7 MB


In [ ]:
X = df.drop(columns=['price'])
y = df['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def objective_xgb(trial):
  learning_rate = trial.suggest_float('learning_rate', 0.1, 0.5)
  n_estimators = trial.suggest_int('n_estimators', 50, 500)
  max_depth = trial.suggest_int('max_depth', 3, 10)

  model = XGBRegressor(
      learning_rate=learning_rate,
      n_estimators=n_estimators,
      max_depth=max_depth,
      objective='reg:squarederror'
  )

  score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
  return np.mean(np.sqrt(-score))

In [ ]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(seed=42))
study.optimize(objective_xgb, n_trials=100)

[I 2024-09-28 19:31:59,787] A new study created in memory with name: no-name-dd09f867-cbea-4eaf-8443-2e3b4ca02541
[I 2024-09-28 19:33:45,750] Trial 0 finished with value: 78464.61127820132 and parameters: {'learning_rate': 0.249816047538945, 'n_estimators': 478, 'max_depth': 8}. Best is trial 0 with value: 78464.61127820132.
[I 2024-09-28 19:34:15,368] Trial 1 finished with value: 72848.32090421933 and parameters: {'learning_rate': 0.3394633936788146, 'n_estimators': 120, 'max_depth': 4}. Best is trial 1 with value: 72848.32090421933.
[I 2024-09-28 19:35:45,860] Trial 2 finished with value: 74534.6857953516 and parameters: {'learning_rate': 0.12323344486727979, 'n_estimators': 440, 'max_depth': 7}. Best is trial 1 with value: 72848.32090421933.
[I 2024-09-28 19:36:10,633] Trial 3 finished with value: 75915.26192839246 and parameters: {'learning_rate': 0.3832290311184182, 'n_estimators': 59, 'max_depth': 10}. Best is trial 1 with value: 72848.32090421933.
[I 2024-09-28 19:36:43,479] Tri

In [ ]:
study.best_params
# {'learning_rate': 0.1783931449676581, 'n_estimators': 70, 'max_depth': 5}

{'learning_rate': 0.1783931449676581, 'n_estimators': 70, 'max_depth': 5}

In [ ]:
params_xgb = {
    'learning_rate': 0.1,
    'n_estimators': 100,
    'max_depth': 8,
    'objective': 'reg:squarederror'
}

# Initialize and train the model
XGB = XGBRegressor(**params_xgb)
# XGB.fit(X, y)

In [ ]:
XGB.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = XGB.predict(X_test)
np.sqrt(mean_squared_error(y_pred, y_test))

81004.58968755358

In [ ]:
## with optuna 2 trials
params_xgb = {'learning_rate': 0.3394633936788146,
              'n_estimators': 120,
              'max_depth': 4,
              'objective': 'reg:squarederror'
}

# Initialize and train the model
XGB = XGBRegressor(**params_xgb)
# XGB.fit(X, y)

In [ ]:
# XGB.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3394633936788146,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=120, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
# y_pred = XGB.predict(X_test)
# np.sqrt(mean_squared_error(y_pred, y_test))

80330.1849110217

In [ ]:
## with 100 optuna trials
params_xgb = {'learning_rate': 0.1783931449676581,
              'n_estimators': 70,
              'max_depth': 5,
              'objective': 'reg:squarederror'
              }

# Initialize and train the model
XGB = XGBRegressor(**params_xgb)
# XGB.fit(X, y)

In [ ]:
# XGB.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1783931449676581,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=70, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
# y_pred = XGB.predict(X_test)
# np.sqrt(mean_squared_error(y_pred, y_test))

80115.32977584527

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
params_cb = {'learning_rate': 0.011601413965844696,
             'iterations': 1668,
             'depth': 8,
             'l2_leaf_reg': 8,
             'bootstrap_type': 'Bayesian',
             'bagging_temperature': 0.7712703466859457,
             'rsm': 0.07404465173409036,
             'loss_function': 'RMSE',
             'eval_metric': 'RMSE',
             'logging_level': 'Silent'
             }
CB = CatBoostRegressor(**params_cb)

In [ ]:
RF = RandomForestRegressor()
# RF.fit(X, y)

In [ ]:
lgbm_params = {
 'objective': 'regression',
 'num_leaves': 20,
 'learning_rate': 0.04309442227534902,
 'n_estimators': 169,
 'feature_fraction': 0.3195906790411752,
 'bagging_fraction': 0.975709499277201,
 'bagging_freq': 4
}
lgb_reg = LGBMRegressor(**lgbm_params)
# lgb_reg.fit(X, y)

In [ ]:
model = VotingRegressor(
    estimators=[('catboost', CB), ('LGB', lgb_reg)],
)
model.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.3195906790411752, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3195906790411752
[LightGBM] [Warning] bagging_fraction is set=0.975709499277201, subsample=1.0 will be ignored. Current value: bagging_fraction=0.975709499277201
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.3195906790411752, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3195906790411752
[LightGBM] [Warning] bagging_fraction is set=0.975709499277201, subsample=1.0 will be ignored. Current value: bagging_fraction=0.975709499277201
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015332 seconds.
Yo

VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7e5f44318520>),
                            ('LGB',
                             LGBMRegressor(bagging_fraction=0.975709499277201,
                                           bagging_freq=4,
                                           feature_fraction=0.3195906790411752,
                                           learning_rate=0.04309442227534902,
                                           n_estimators=169, num_leaves=20,
                                           objective='regression'))])

In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Squared Error: {np.sqrt(mse):.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.2f}")

[LightGBM] [Warning] feature_fraction is set=0.3195906790411752, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3195906790411752
[LightGBM] [Warning] bagging_fraction is set=0.975709499277201, subsample=1.0 will be ignored. Current value: bagging_fraction=0.975709499277201
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Root Mean Squared Error: 77178.31
Mean Absolute Error: 19273.33
R² Score: 0.13


In [ ]:
test = pd.read_csv('/content/test.csv')

for column in test.columns:
  if test[column].dtype == 'object':
    test[column].fillna(test[column].mode()[0], inplace=True)
  else:
    test[column].fillna(test[column].mean(), inplace=True)
test.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,Yes
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,Yes
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [ ]:
test = encode(test)

In [ ]:
y_pred = model.predict(test)

# Create a DataFrame for the submission
submission_df = pd.DataFrame({'id': test['id'], 'price': y_pred.flatten()})
submission_df['price'] = submission_df['price'].astype(int)

# Save the submission to a CSV file
submission_df.to_csv('submission.csv', index=False)

[LightGBM] [Warning] feature_fraction is set=0.3195906790411752, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3195906790411752
[LightGBM] [Warning] bagging_fraction is set=0.975709499277201, subsample=1.0 will be ignored. Current value: bagging_fraction=0.975709499277201
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


## This is the best model so far and scored on RMSE: 73820.19488

### Competition link:
https://www.kaggle.com/competitions/playground-series-s4e9